In [1]:
import os
import json
import pandas as pd
import traceback

In [4]:
# %pip install langchain-community

from langchain.chat_models import ChatOpenAI

In [7]:
from dotenv import load_dotenv

load_dotenv()

KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(
    openao_api_key=,
    model="gpt-4",
    temperature=0.5
)